# Module 7 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-01-programming.ipynb
    +--module-01-programming.html
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID.

Imports here if needed.

In [1]:
import copy, bisect

# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [8]:
from unification import parse, unification, unify

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that. **(but please please do not try to implement STRIPS or GraphPlan...that is wrong).**

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Store)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones. Trust me on this.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". Where?

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester appliesm, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...but if you pass an extra debug=True parameter, it should also return the intermediate *states* as well as the actions.

-----

### Helper Functions ###

**applySubs(xprsn, sub)**  
This function applies the substitution to the expression in the argument. The algorithm goes over each parts of the expression, and replace any variable with a constant if the variable has a substitution. If the part of the expression is another nested expression, it recurs into it and apply the same substitution.

In [109]:
def applySubs(expr, sub):
    return [applySubs(t,sub) if type(t)==list \
            else sub[t] if t in sub else t for t in expr]

def multiVarSameSub(estab, new):
    invEstab = dict((v, k) for k, v in estab.items())
    return any([invEstab[v]!=k for k,v in new.items() if v in invEstab])

################################################################################
def getActions(inConds, inState):
    acts = list()
    if len(inConds) > len(inState):
        return acts # must have more states than preconditions

    state = [parse(s) for s in inState]
    p = [ ({}, [parse(c) for c in inConds]) ]
    while len(p) > 0:
        sub, conds = p.pop()
        #print('Popped: ' + repr((sub, conds)))
        for n,s in enumerate(state):
            #print('\tUnifying %s with state %s'%(repr(conds[0]),repr(s)))
            tmp = unification(conds[0],s)
            #print('\t\tResult: %s'%repr(tmp))
            if tmp != False and not multiVarSameSub(sub,tmp):
                tmp.update(sub)
                if len(conds) == 1:
                    acts.append(tmp)
                else:
                    #print('\tPushing: %s and %s'%(repr(tmp),repr(conds[1:])))
                    p.append((tmp, [applySubs(c,tmp) for c in conds[1:]]))
            #print('')
    return acts


In [112]:
#print multiVarSameSub({'?x':'cup'}, {'?x':'cup'})

conds = ['(on_table ?x)', '(empty ?y)']
st = ['(on_table cup)', '(on_table bowl)', '(empty cup)', '(empty bowl)']
#print getActions(conds, st)

conds2 = ["(agent ?agent)",
         "(place ?from)",
         "(place ?to)",
         "(at ?agent ?from)"]
st2 = ["(item Drill)",
       "(place Home)",
       "(place Store)",
       "(agent Me)", 
       "(at Me Home)",
       "(at Drill Store)"]
#print getActions(conds2, st2)

conds3 = ['(on_table ?x)', '(empty ?y)']
st3 = ['(on_table cup)', '(on_table bowl)']
print getActions(conds3, st3)

[]


(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


-----

So you need to implement `forward_planner` as described above. `start_state`, `goal` and `actions` should all have the layout above and be s-expressions.

Your implementation should return the plan as a **List of instantiated actions**. If `debug=True`, you should print out the intermediate states of the plan as well.

In [ ]:
def forward_planner( start_state, goal, actions, debug=False):
    return []

You will be solving the problem from above. Here is the start state:

In [ ]:
start_state = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Store)"
]

The goal state:

In [ ]:
goal = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)"
]

and the actions/operators:

In [ ]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

In [ ]:
plan = forward_planner( start_state, goal, actions)
print plan

In [ ]:
plan_with_states = forward_planner( start_state, goal, actions, debug=True)
print plan_with_states